## **Retail Store Sales**

**Overview**

The Dirty Retail Store Sales dataset contains 12,575 rows of synthetic data representing sales transactions from a retail store. The dataset includes eight product categories with 25 items per category, each having static prices. It is designed to simulate real-world sales data, including intentional "dirtiness" such as missing or inconsistent values. This dataset is suitable for practicing data cleaning, exploratory data analysis (EDA), and feature engineering.


# Cleaning and Validating Retail Store Sales Data

This notebook demonstrates **data cleaning, validation, feature engineering, and exploratory data analysis (EDA)** on a retail store sales dataset containing **12,575 transactions** with 11 features.  

The dataset includes transaction-level details such as **customer information, product categories, item descriptions, prices, quantities, payment methods, store locations, and transaction dates**. It also contains **7,229 missing values** and several inconsistencies that make it an excellent case study for practical data preparation.

---

### Data Cleaning and Validation
- **Handling missing values** – addressing the 7,229 null values with appropriate imputation or removal strategies.  
- **Data type corrections** – converting `Transaction Date` to datetime, ensuring numerical columns (`Price Per Unit`, `Quantity`, `Total Spent`) have consistent precision, and setting categorical fields (`Category`, `Payment Method`, `Location`).  
- **Validation checks** – detecting duplicate `Transaction ID`s, invalid prices or quantities (e.g., negative values), and mismatched totals between `Quantity × Price Per Unit` and `Total Spent`.  
- **Normalising categorical variables** – standardising spellings, capitalisation, and encoding categories for analysis.  
- **Ensuring data integrity** – verifying unique IDs, cross-checking totals, and validating customer-transaction relationships.  

---

### Exploratory Data Analysis (EDA)
After cleaning, we will extract insights such as:  
- **Descriptive statistics** for sales volumes, revenues, and product distribution.  
- **Univariate and bivariate analysis** of product categories, payment methods, and store locations.  
- **Time series analysis** using `Transaction Date` to uncover seasonal patterns and monthly sales trends.  
- **Customer behaviour analysis** – e.g., average spend per customer, popular items, and repeat purchases.  
- **Visualisation** with bar charts, line plots, histograms, and heatmaps to communicate findings clearly.  

---

This workflow not only prepares the dataset for **machine learning pipelines** but also generates **business insights** to guide decision-making in retail strategy, inventory planning, and customer engagement.


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

filename = "retail_store_sales.csv"

#Load Data file 
sales_data = pd.read_csv(filename)

#Quick checking data 
print(sales_data.shape)
print(f"\n{sales_data.head()}")
print(sales_data.info())


(12575, 11)

  Transaction ID Customer ID       Category          Item  Price Per Unit  \
0    TXN_6867343     CUST_09     Patisserie   Item_10_PAT            18.5   
1    TXN_3731986     CUST_22  Milk Products  Item_17_MILK            29.0   
2    TXN_9303719     CUST_02       Butchers   Item_12_BUT            21.5   
3    TXN_9458126     CUST_06      Beverages   Item_16_BEV            27.5   
4    TXN_4575373     CUST_05           Food   Item_6_FOOD            12.5   

   Quantity  Total Spent  Payment Method Location Transaction Date  \
0      10.0        185.0  Digital Wallet   Online       2024-04-08   
1       9.0        261.0  Digital Wallet   Online       2023-07-23   
2       2.0         43.0     Credit Card   Online       2022-10-05   
3       9.0        247.5     Credit Card   Online       2022-05-07   
4       7.0         87.5  Digital Wallet   Online       2022-10-02   

  Discount Applied  
0             True  
1             True  
2            False  
3              NaN 

In [6]:
#Cread data copy 
Data_copy = sales_data.copy()

In [7]:
#Standardization columns name 
col = ["Category", "Item", "Payment Method", "Location"]
Data_copy[col] = Data_copy[col].apply(lambda x : x.str.strip()
                                                    .str.title()
                                                    .str.replace("_"," "))

Data_copy.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item 10 Pat,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item 17 Milk,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item 12 But,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item 16 Bev,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item 6 Food,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


## Checking NaN data in dataset

- After standardizing the categorical columns, check the entire dataset for missing values (NaN).


In [10]:
missing = Data_copy.isna().sum()[Data_copy.isna().sum() > 0]
missing_values = pd.DataFrame({"Na_Values": missing, "Na_Percentage": missing/len(sales_data)*100}).sort_values(by="Na_Percentage", ascending=False)

missing_values

,Na_Values,Na_Percentage
Discount Applied,4199,33.391650
Item,1213,9.646123
Price Per Unit,609,4.842942
Quantity,604,4.803181
Total Spent,604,4.803181


# Missing Values

The dataset contains several missing values across different columns:

- **Discount Applied**: 4,199 missing values (**33.39%**)  
- **Item**: 1,213 missing values (**9.64%**)  
- **Price Per Unit**: 609 missing values (**4.84%**)  
- **Quantity**: 604 missing values (**4.80%**)  
- **Total Spent**: 604 missing values (**4.80%**)  

Among these, the highest proportion of missing values is in the  
**Discount Applied** column (**33.39%**).  

- Since this column only indicates whether a discount was applied  
  (`True` / `False`), missing entries can reasonably be assumed as `False`.  

- For the other columns, further treatment strategies will be applied,  
  such as:  
  - Cross-checking with related features  
  - Removing incomplete rows where necessary



In [52]:
print(f"Discount Appplied: {Data_copy["Discount Applied"].unique()}") 
print(f"Total Value Before Clean : {Data_copy["Discount Applied"].nunique()}")

#imputation Na as False
Data_copy["Discount Missing"] = Data_copy["Discount Applied"].isna()
Data_copy["Discount Applied"] = Data_copy["Discount Applied"].fillna(False)

print(f"Discount Applied:{Data_copy["Discount Applied"].unique()}")

Data_copy.drop(columns="Discount Missing",inplace=True)

Discount Appplied: [ True False]
Total Value Before Clean : 2
Discount Applied:[ True False]


# Checking Missing Value "Item"

In [27]:
Data_copy.groupby("Category")["Item"].agg(
    total="size",
    missing=lambda x: x.isna().sum(),
    missing_pct=lambda x: round(x.isna().mean()*100,2)
)

,total,missing,missing_pct
Category,,,
Beverages,1567,140,8.93
Butchers,1568,147,9.38
Computers And Electric Accessories,1558,161,10.33
Electric Household Essentials,1591,154,9.68
Food,1588,162,10.20
Furniture,1591,131,8.23
Milk Products,1584,159,10.04
Patisserie,1528,159,10.41


### Mapping Item Codes  

The `Item` column originally contained product codes (e.g., `Item_1_EHE`, `Item_2_FUR`),  
which were not intuitive for analysis. To improve readability, these codes were mapped  
to their corresponding **product names** and **prices** using a predefined lookup table.  

Two new columns were added:  
- `Item_Name` → product name.  
- `Item_Price` → standard item price.  

This makes the dataset clearer and easier to interpret in business terms.

| Item Code     | Item Name         | Price |
|---------------|------------------|-------|
| Item_1_EHE    | Blender          | 5.0   |
| Item_2_EHE    | Microwave        | 6.5   |
| Item_3_EHE    | Toaster          | 8.0   |
| Item_4_EHE    | Vacuum Cleaner   | 9.5   |
| Item_5_EHE    | Air Purifier     | 11.0  |
| Item_6_EHE    | Electric Kettle  | 12.5  |
| Item_7_EHE    | Rice Cooker      | 14.0  |
| Item_8_EHE    | Iron             | 15.5  |
| Item_9_EHE    | Ceiling Fan      | 17.0  |
| Item_10_EHE   | Table Fan        | 18.5  |
| Item_11_EHE   | Hair Dryer       | 20.0  |
| Item_12_EHE   | Heater           | 21.5  |
| Item_13_EHE   | Humidifier       | 23.0  |
| Item_14_EHE   | Dehumidifier     | 24.5  |
| Item_15_EHE   | Coffee Maker     | 26.0  |
| Item_16_EHE   | Portable AC      | 27.5  |
| Item_17_EHE   | Electric Stove   | 29.0  |
| Item_18_EHE   | Pressure Cooker  | 30.5  |
| Item_19_EHE   | Induction Cooktop| 32.0  |
| Item_20_EHE   | Water Dispenser  | 33.5  |
| Item_21_EHE   | Hand Blender     | 35.0  |
| Item_22_EHE   | Mixer Grinder    | 36.5  |
| Item_23_EHE   | Sandwich Maker   | 38.0  |
| Item_24_EHE   | Air Fryer        | 39.5  |
| Item_25_EHE   | Juicer           | 41.0  |


In [63]:
#cread copy for dataset 
data_copy = Data_copy.copy()

df_elec = data_copy[data_copy["Category"] == "Computers And Electric Accessories"]
df_elec

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
46,TXN_6404316,CUST_04,Computers And Electric Accessories,Item 16 Cea,27.5,6.0,165.0,Credit Card,Online,2022-04-06,False
51,TXN_8413369,CUST_23,Computers And Electric Accessories,Item 5 Cea,11.0,6.0,66.0,Credit Card,Online,2024-03-09,True
56,TXN_7909783,CUST_12,Computers And Electric Accessories,Item 5 Cea,11.0,1.0,11.0,Credit Card,Online,2023-12-17,True
57,TXN_5046007,CUST_07,Computers And Electric Accessories,Item 15 Cea,26.0,9.0,234.0,Cash,Online,2024-07-22,True
60,TXN_6820452,CUST_10,Computers And Electric Accessories,Item 24 Cea,39.5,8.0,316.0,Credit Card,In-Store,2023-04-18,False
...,...,...,...,...,...,...,...,...,...,...,...
12534,TXN_9556376,CUST_04,Computers And Electric Accessories,Item 11 Cea,20.0,4.0,80.0,Credit Card,Online,2023-07-11,False
12544,TXN_6726738,CUST_04,Computers And Electric Accessories,Item 10 Cea,18.5,3.0,55.5,Credit Card,In-Store,2022-09-24,False
12553,TXN_3271634,CUST_25,Computers And Electric Accessories,Item 15 Cea,26.0,4.0,104.0,Credit Card,Online,2023-02-26,True
12561,TXN_6960333,CUST_12,Computers And Electric Accessories,Item 17 Cea,29.0,2.0,58.0,Credit Card,Online,2024-12-12,True


## Input Name of Item
- input new columns for items name 

In [66]:
#load data dict Item 
dict_item = pd.read_csv("Item_dict.csv")
dict_item.head()

    

,Item Code,Item Name,Price
0,Item 1 Cea,Blender,5.0
1,Item 2 Cea,Microwave,6.5
2,Item 3 Cea,Toaster,8.0
3,Item 4 Cea,Vacuum Cleaner,9.5
4,Item 5 Cea,Air Purifier,11.0


## Merging Item Dictionary and Handling Price Columns

- First, we rename the column **Item Code** to **Item** in `dict_item` for consistency.  
- Then, we merge `data_copy` with `dict_item` using **Item** as the key (`how="left"`).  
- After merging, we handle duplicate price columns:  
  - Fill missing values in **Price Per Unit** with values from **Price**.  
  - Drop the redundant **Price** column to avoid duplication.  
- As a result, the final dataset keeps only the **Price Per Unit** column with completed values.  



In [90]:
dict_item = dict_item.rename(columns={"Item Code": "Item"})

data_merge = data_copy.merge(dict_item, on="Item", how="left")

data_merge["Price Per Unit"] = data_merge["Price Per Unit"].fillna(data_merge["Price"]) 

data_merge.drop(columns="Price", inplace=True)

data_merge




,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Item Name
0,TXN_6867343,CUST_09,Patisserie,Item 10 Pat,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True,NaN
1,TXN_3731986,CUST_22,Milk Products,Item 17 Milk,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True,NaN
2,TXN_9303719,CUST_02,Butchers,Item 12 But,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False,NaN
3,TXN_9458126,CUST_06,Beverages,Item 16 Bev,27.5,9.0,247.5,Credit Card,Online,2022-05-07,False,NaN
4,TXN_4575373,CUST_05,Food,Item 6 Food,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item 23 Pat,38.0,4.0,152.0,Credit Card,In-Store,2023-09-03,False,NaN
12571,TXN_4009414,CUST_03,Beverages,Item 2 Bev,6.5,9.0,58.5,Cash,Online,2022-08-12,False,NaN
12572,TXN_5306010,CUST_11,Butchers,Item 7 But,14.0,10.0,140.0,Cash,Online,2024-08-24,False,NaN
12573,TXN_5167298,CUST_04,Furniture,Item 7 Fur,14.0,6.0,84.0,Cash,Online,2023-12-30,True,Wardrobe


## Enriching "Price Per Unit" and Filling Missing Items

- First, we calculate a new variable **Item_price** by dividing **Total Spent** by **Quantity**.  
  - This step helps to estimate missing values in **Price Per Unit** based on available transaction data.  

- Next, we fill missing values in **Price Per Unit** using the calculated **Item_price**.  
  - This ensures that every transaction has a valid unit price, even if it was missing in the original dataset.  

- After that, we create a **mapping** from `(Category, Price Per Unit) → Item` using rows where the Item name is available.  
  - This mapping acts as a reference to identify items when the Item column is missing.  

- Finally, we update the **Item** column by applying the mapping:
  - If an Item is missing but the combination `(Category, Price Per Unit)` is already known, the missing Item will be filled automatically.  
  - If the combination does not exist in the mapping, the value remains unchanged.  

### Why we do this?
- To **recover missing values** in both price and item columns using logical relationships in the data.  
- To make the dataset **more consistent and reliable** for analysis by reducing NaN values.  
- To ensure that transactions with the same **Category + Price Per Unit** are mapped to the same **Item**, improving data integrity.  


In [100]:
Item_price = data_merge["Total Spent"] / data_merge["Quantity"]

data_merge["Price Per Unit"] = data_merge["Price Per Unit"].fillna(Item_price)

mapping = (data_merge.dropna(subset=["Item"])
    .set_index(["Category","Price Per Unit"])["Item"]
    .to_dict()
)

data_merge["Item"] = data_merge.apply(
    lambda row: mapping.get((row["Category"], row["Price Per Unit"]), row["Item"]),
    axis=1
)


## Checking Item Imputation

- We select a specific transaction (`Transaction ID = TXN_7068372`) from the **imputed dataset** (`data_merge`) and the **original dataset** (`data_copy`).  
- This comparison allows us to verify whether the **Item Imputation process** has been applied correctly.  
- By reviewing the two subsets (`df_elec` and `df_elec1`), we can clearly see how missing values were filled and ensure that the logic of imputation works as expected.  


In [111]:
#checking Item Imputation
df_elec = data_merge[data_merge["Transaction ID"] == "TXN_7068372"]
df_elec1 = data_copy[data_copy["Transaction ID"] == "TXN_7068372"]

df_elec

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Item Name
784,TXN_7068372,CUST_23,Computers And Electric Accessories,Item 13 Cea,23.0,NaN,NaN,Digital Wallet,In-Store,2022-12-10,False,NaN


In [ ]:
#checking Item before imputation
df_elec1

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
784,TXN_7068372,CUST_23,Computers And Electric Accessories,NaN,23.0,NaN,NaN,Digital Wallet,In-Store,2022-12-10,False


## Handling Missing Values in "Quantity" & "Total Spending"

- First, we create a new variable `total` by multiplying **Quantity** and **Price Per Unit**.  
- Next, we perform **imputation** for the column **Total Spent** using the calculated `total`.  
- Finally, we check the dataset again to see the **remaining missing values** with `data_merge.isna().sum()`.  


In [ ]:
#Cread varible for checking data
total = data_merge["Quantity"] * data_merge["Price Per Unit"]
#imputasi Total Spent with Total
data_merge["Total Spent"] = data_merge["Total Spent"].fillna(total)
#Checking Missing Values
data_merge.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied       0
Item Name           9718
dtype: int64

### Missing Value Analysis

- As we can see, **Quantity** has the same number of missing values.  
- We are checking missing values with **imputation** to understand the data better.  
- All missing values in **Quantity** are the same as in **Total Spent**, which indicates a direct relationship between them.  
- Finally, we decide to **drop missing values** because when both **Quantity** and **Total Spent** are empty, the data cannot be used.  
- Moreover, the percentage of missing data is still below **5%**, so dropping them will not significantly impact the dataset.  


In [121]:
data_merge.dropna(subset=["Quantity", "Total Spent"], inplace=True)
data_merge.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied       0
Item Name           9114
dtype: int64

## Checking Tipe data "Transaction Date"

In [132]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11971 entries, 0 to 12574
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    11971 non-null  object        
 1   Customer ID       11971 non-null  object        
 2   Category          11971 non-null  object        
 3   Item              11971 non-null  object        
 4   Price Per Unit    11971 non-null  float64       
 5   Quantity          11971 non-null  float64       
 6   Total Spent       11971 non-null  float64       
 7   Payment Method    11971 non-null  object        
 8   Location          11971 non-null  object        
 9   Transaction Date  11971 non-null  datetime64[ns]
 10  Discount Applied  11971 non-null  bool          
 11  Item Name         2857 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(3), object(7)
memory usage: 1.1+ MB


In [135]:
data_merge["Transaction Date"] = pd.to_datetime(data_merge["Transaction Date"], yearfirst=True, errors="coerce")
# Feature 1: transaction_month
data_merge["Transaction month "] = data_merge["Transaction Date"].dt.strftime('%B')  # January, February, etc.
# Feature 2: day_of_the_week
data_merge["day of the week"] = data_merge["Transaction Date"].dt.day_name()  # Monday, Tuesday, etc.

data_merge


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Item Name,Transaction month,day of the week
0,TXN_6867343,CUST_09,Patisserie,Item 10 Pat,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True,NaN,April,Monday
1,TXN_3731986,CUST_22,Milk Products,Item 17 Milk,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True,NaN,July,Sunday
2,TXN_9303719,CUST_02,Butchers,Item 12 But,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False,NaN,October,Wednesday
3,TXN_9458126,CUST_06,Beverages,Item 16 Bev,27.5,9.0,247.5,Credit Card,Online,2022-05-07,False,NaN,May,Saturday
4,TXN_4575373,CUST_05,Food,Item 6 Food,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False,NaN,October,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item 23 Pat,38.0,4.0,152.0,Credit Card,In-Store,2023-09-03,False,NaN,September,Sunday
12571,TXN_4009414,CUST_03,Beverages,Item 2 Bev,6.5,9.0,58.5,Cash,Online,2022-08-12,False,NaN,August,Friday
12572,TXN_5306010,CUST_11,Butchers,Item 7 But,14.0,10.0,140.0,Cash,Online,2024-08-24,False,NaN,August,Saturday
12573,TXN_5167298,CUST_04,Furniture,Item 7 Fur,14.0,6.0,84.0,Cash,Online,2023-12-30,True,Wardrobe,December,Saturday


In [136]:
data_merge.to_csv("Sales Data Retail Store.csv",index=True)